# Neural Net and CNN Exploration of Data

In [8]:
# Train a NN model on the dataset

# Load the dataset
import pandas as pd

data = pd.read_csv('../normalized_data.csv')
data = data[['Social support', 'Log GDP per capita', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Happiness_Score_Percentile']]
data = data.dropna()

In [9]:
print(min(data['Happiness_Score_Percentile']))

0


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Define the model
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        # self.fc3 = nn.Linear(13, 13)
        self.fc2 = nn.Linear(10, 10)
        # self.fc2 = nn.Linear(8, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        # x = self.fc3(x)
        # x = torch.relu(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

In [42]:
# Make a CNN model with input of size (931, 4) and output of size 10

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(4, 10, 1)
        self.max_pool1 = nn.MaxPool1d(1)
        self.conv2 = nn.Conv1d(10, 10, 1)
        self.max_pool2 = nn.MaxPool1d(1)
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.max_pool1(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.max_pool2(x)
        x = self.dropout(x)
        x = x.view(-1, 10)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.dropout(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output



In [11]:
# Split the data into training and testing sets
labels = data['Happiness_Score_Percentile']
data.drop(['Happiness_Score_Percentile'], axis=1, inplace=True)
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)
# val_data, test_data, val_labels, test_labels = train_test_split(test_data, test_labels, test_size=0.5, random_state=42)

# Convert the data to tensors
train_data = torch.tensor(train_data.values, dtype=torch.float32)
train_labels = torch.tensor(train_labels.values, dtype=torch.long)
# val_data = torch.tensor(val_data.values, dtype=torch.float32)
# val_labels = torch.tensor(val_labels.values, dtype=torch.long)
test_data = torch.tensor(test_data.values, dtype=torch.float32)
test_labels = torch.tensor(test_labels.values, dtype=torch.long)

In [51]:
# Create the model
model = NN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

# Train the model
epochs = 3000
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(train_data)
    loss = criterion(output, train_labels)
    loss.backward()
    optimizer.step()
    print('Epoch: {} - Loss: {:.6f}'.format(epoch, loss.item()))

# Test the model
correct = 0
total = 0
with torch.no_grad():
    output = model(test_data)
    for idx, i in enumerate(output):
        if torch.argmax(i) == test_labels[idx]:
            correct += 1
        total += 1
print('Accuracy: ', round(correct/total, 3))

Epoch: 0 - Loss: 2.465357
Epoch: 1 - Loss: 2.416625
Epoch: 2 - Loss: 2.372779
Epoch: 3 - Loss: 2.333263
Epoch: 4 - Loss: 2.297561
Epoch: 5 - Loss: 2.265308
Epoch: 6 - Loss: 2.235938
Epoch: 7 - Loss: 2.208717
Epoch: 8 - Loss: 2.182959
Epoch: 9 - Loss: 2.158240
Epoch: 10 - Loss: 2.134243
Epoch: 11 - Loss: 2.110792
Epoch: 12 - Loss: 2.087788
Epoch: 13 - Loss: 2.065157
Epoch: 14 - Loss: 2.042923
Epoch: 15 - Loss: 2.021116
Epoch: 16 - Loss: 1.999765
Epoch: 17 - Loss: 1.978888
Epoch: 18 - Loss: 1.958542
Epoch: 19 - Loss: 1.938689
Epoch: 20 - Loss: 1.919322
Epoch: 21 - Loss: 1.900459
Epoch: 22 - Loss: 1.882149
Epoch: 23 - Loss: 1.864435
Epoch: 24 - Loss: 1.847433
Epoch: 25 - Loss: 1.831133
Epoch: 26 - Loss: 1.815525
Epoch: 27 - Loss: 1.800598
Epoch: 28 - Loss: 1.786357
Epoch: 29 - Loss: 1.772842
Epoch: 30 - Loss: 1.760064
Epoch: 31 - Loss: 1.748004
Epoch: 32 - Loss: 1.736625
Epoch: 33 - Loss: 1.725927
Epoch: 34 - Loss: 1.715802
Epoch: 35 - Loss: 1.706185
Epoch: 36 - Loss: 1.697003
Epoch: 37 -

In [49]:
model = CNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

epochs = 5000
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(train_data.view(-1, 4, 1))
    loss = criterion(output, train_labels)
    loss.backward()
    optimizer.step()
    print('Epoch: {} - Loss: {:.6f}'.format(epoch, loss.item()))

# Test the model
correct = 0
total = 0
with torch.no_grad():
    output = model(test_data.view(-1, 4, 1))
    for idx, i in enumerate(output):
        if torch.argmax(i) == test_labels[idx]:
            correct += 1
        total += 1
print('Accuracy: ', round(correct/total, 3))
    

Epoch: 0 - Loss: 2.301803
Epoch: 1 - Loss: 2.284664
Epoch: 2 - Loss: 2.270281
Epoch: 3 - Loss: 2.249685
Epoch: 4 - Loss: 2.233288
Epoch: 5 - Loss: 2.225379
Epoch: 6 - Loss: 2.201171
Epoch: 7 - Loss: 2.193288
Epoch: 8 - Loss: 2.159131
Epoch: 9 - Loss: 2.153384
Epoch: 10 - Loss: 2.114380
Epoch: 11 - Loss: 2.115511
Epoch: 12 - Loss: 2.108809
Epoch: 13 - Loss: 2.079052
Epoch: 14 - Loss: 2.064611
Epoch: 15 - Loss: 2.007514
Epoch: 16 - Loss: 2.006155
Epoch: 17 - Loss: 1.975967
Epoch: 18 - Loss: 1.946495
Epoch: 19 - Loss: 1.950683
Epoch: 20 - Loss: 1.933763
Epoch: 21 - Loss: 1.917502
Epoch: 22 - Loss: 1.949382
Epoch: 23 - Loss: 1.919574
Epoch: 24 - Loss: 1.909187
Epoch: 25 - Loss: 1.925170
Epoch: 26 - Loss: 1.907568
Epoch: 27 - Loss: 1.918546
Epoch: 28 - Loss: 1.864107
Epoch: 29 - Loss: 1.848197
Epoch: 30 - Loss: 1.882559
Epoch: 31 - Loss: 1.831585
Epoch: 32 - Loss: 1.831455
Epoch: 33 - Loss: 1.849464
Epoch: 34 - Loss: 1.849041
Epoch: 35 - Loss: 1.868847
Epoch: 36 - Loss: 1.843894
Epoch: 37 -